# ADS-508-01-SP23 Team 8: Final Project

# Train model

Much of the code is modified from `Fregly, C., & Barth, A. (2021). Data science on AWS: Implementing end-to-end, continuous AI and machine learning pipelines. O’Reilly.`

## Install missing dependencies

[PyAthena](https://pypi.org/project/PyAthena/) is a Python DB API 2.0 (PEP 249) compliant client for Amazon Athena.

In [2]:
!pip install --disable-pip-version-check -q PyAthena==2.1.0
!pip install --disable-pip-version-check -q sagemaker-experiments==0.1.26
!pip install missingno
!pip install scikit-optimize

## Globally import libraries

In [3]:
import boto3
from botocore.client import ClientError
import pandas as pd
import numpy as np
from pyathena import connect
from IPython.core.display import display, HTML
import missingno as msno
from skopt import BayesSearchCV
from skopt.space import Real, Categorical, Integer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.impute import SimpleImputer
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.linear_model import Lasso
import datetime as dt
import time
import sagemaker
from sagemaker.sklearn.estimator import SKLearn
from smexperiments.experiment import Experiment
from smexperiments.trial import Trial
import joblib
import os
from io import BytesIO

## Instantiate AWS SageMaker and S3 sessions

In [12]:
session = boto3.session.Session()
role = sagemaker.get_execution_role()
region = session.region_name
sagemaker_session = sagemaker.Session()
def_bucket = sagemaker_session.default_bucket()
bucket = 'sagemaker-us-east-ads508-sp23-t8'

s3 = boto3.Session().client(service_name="s3",
                            region_name=region)

sm = boto3.Session().client(service_name="sagemaker",
                            region_name=region)

print(role)
print(region)
print( boto3.Session().region_name)


arn:aws:iam::657724983756:role/LabRole
us-east-1
us-east-1
Region: us-east-1


In [5]:
setup_s3_bucket_passed = False
ingest_create_athena_db_passed = False
ingest_create_athena_table_tsv_passed = False

In [6]:
print(f"Default bucket: {def_bucket}")
print(f"Public T8 bucket: {bucket}")

Default bucket: sagemaker-us-east-1-657724983756
Public T8 bucket: sagemaker-us-east-ads508-sp23-t8


## Pass in train and test X from CSV

In [7]:
s3_train_path = f"s3://{def_bucket}/team_8_data/modeling_data/training"
s3_test_path = f"s3://{def_bucket}/team_8_data/modeling_data/testing"

s3_train_x01_csv_path = f"s3://{def_bucket}/team_8_data/modeling_data/training/train_x01.csv"
train_x01 = pd.read_csv(s3_train_x01_csv_path)
s3_test_x01_csv_path = f"s3://{def_bucket}/team_8_data/modeling_data/testing/test_x01.csv"
test_x01 = pd.read_csv(s3_test_x01_csv_path)

print(s3_train_x01_csv_path)
print(s3_test_x01_csv_path)


print(f'{train_x01.shape}')
print(f'\n{test_x01.shape}')

s3://sagemaker-us-east-1-657724983756/team_8_data/modeling_data/training/train_x01.csv
s3://sagemaker-us-east-1-657724983756/team_8_data/modeling_data/testing/test_x01.csv
(25284, 48)

(6321, 48)


## Pass in train and test y from np array

train_X_uri = sagemaker_session.upload_data('train_x01.csv', bucket=def_bucket)
train_y_uri = sagemaker_session.upload_data('train_y01.npy', bucket=def_bucket)

print(train_X_uri)
print(train_y_uri)

In [8]:
# Define the S3 object key
train_y01_s3_key = 'team_8_data/modeling_data/training/train_y01.npy'

# Load the numpy array from S3
with BytesIO() as data:
    s3.download_fileobj(def_bucket, train_y01_s3_key, data)
    data.seek(0)
    train_y01 = np.load(data)

# Define the S3 object key
test_y01_s3_key = 'team_8_data/modeling_data/testing/test_y01.npy'

# Load the numpy array from S3
with BytesIO() as data:
    s3.download_fileobj(def_bucket, test_y01_s3_key, data)
    data.seek(0)
    test_y01 = np.load(data)

# Confirm that the numpy array was loaded from S3
print(f'{train_y01.shape}')
print(f'{test_y01.shape}')

(25284, 1)
(6321, 1)


## Model Training using Grid search with 5-fold cross-validation

### Neural Network

# Start timer script
start_time = dt.datetime.today()

# Citation: Hochberg, 2018; Shanmukh, 2021
m2v1_pip = Pipeline([('si', SimpleImputer(strategy='median')),
                     ('ss', StandardScaler()),
                     ('nn', MLPRegressor(random_state=1699))])

nodes_h = 3
predictors_p = 49

hidden_layer_sizes_hparam = [[100,],
                             [(nodes_h*(predictors_p+1))+nodes_h+1,],
                             [50, 50]
                            ]
activation_hparam = ['logistic', 'relu']
solver_hparam = ['adam']
alpha_hparam = [.0001, .0005, .001]
learn_rate_hparam = ['constant', 'invscaling']

#hidden_layer_sizes_hparam = [[100,]]
#activation_hparam = ['relu']
#solver_hparam = ['adam']
#alpha_hparam = [.0001]
#learn_rate_hparam = ['invscaling']

m2v1_grd = {'nn__hidden_layer_sizes': hidden_layer_sizes_hparam,
            'nn__activation': activation_hparam,
            'nn__solver': solver_hparam,
            'nn__alpha': alpha_hparam,
            'nn__learning_rate': learn_rate_hparam
           }

m2v1_nn = GridSearchCV(m2v1_pip,
                       m2v1_grd,
                       scoring='neg_root_mean_squared_error',
                       n_jobs=2,
                       refit=True,
                       verbose=2)

m2v1_nn.fit(train_x01, train_y01)

print(f'Best Estimator:\n{m2v1_nn.best_estimator_}')

print(pd.DataFrame(m2v1_nn.cv_results_))

train_m2v1_y01_pred = m2v1_nn.predict(train_x01)
print(train_m2v1_y01_pred)

test_m2v1_y01_pred = m2v1_nn.predict(test_x01)
print(test_m2v1_y01_pred)

# Display evaluation metrics
# R-sq
train_m2v1_nn_r2 = r2_score(train_y01, train_m2v1_y01_pred)
test_m2v1_nn_r2 = r2_score(test_y01, test_m2v1_y01_pred)

print(f'Train R-sq:\n{train_m2v1_nn_r2}')
print(f'Test R-sq:\n{test_m2v1_nn_r2}')

# RMSE
train_m2v1_nn_rmse = mean_squared_error(train_y01, train_m2v1_y01_pred, squared=False)
test_m2v1_nn_rmse = mean_squared_error(test_y01, test_m2v1_y01_pred, squared=False)

print(f'Train RMSE:\n{train_m2v1_nn_rmse}')
print(f'Test RMSE:\n{test_m2v1_nn_rmse}')

# End timer script
end_time = dt.datetime.today()
time_elapse = end_time - start_time
print(f'End Time = {end_time}')
print(f'Script Time = {time_elapse}')

s3_m2v1_nn_pqt_base_path = f"../models"

if not os.path.exists(s3_m2v1_nn_pqt_base_path):
    os.makedirs(s3_m2v1_nn_pqt_base_path)

s3_m2v1_nn_pqt_path = os.path.join(s3_m2v1_nn_pqt_base_path,
                                   'm2v1_nn.parquet')

# save the model to disk using joblib
joblib.dump(m2v1_nn,
            s3_m2v1_nn_pqt_path)

# load the saved model from disk using joblib
m2v1_nn_fitted = joblib.load(s3_m2v1_nn_pqt_path)

### Lasso - Using `GridSearchCV`

# Start timer script
start_time = dt.datetime.today()

# Citation: Hochberg, 2018; Shanmukh, 2021
m3v1_ls_pip = Pipeline([('si', SimpleImputer(strategy='median')),
                        ('ss', StandardScaler()),
                        ('ls', Lasso(random_state=1699))])

alpha_hparam = [.01, .05, .1, .5, 1, 2]
selection_hparam = ['cyclic', 'random']


m3v1_ls_grd = {'ls__alpha': alpha_hparam,
               'ls__selection': selection_hparam
           }

m3v1_ls = GridSearchCV(m3v1_ls_pip,
                       m3v1_ls_grd,
                       scoring='neg_root_mean_squared_error',
                       n_jobs=2,
                       refit=True,
                       verbose=2)

m3v1_ls.fit(train_x01, train_y01)

print(f'Best Estimator:\n{m3v1_ls.best_estimator_}')
print(f'Coefficients:\n{m3v1_ls.best_estimator_.named_steps["ls"].coef_}')

print(pd.DataFrame(m3v1_ls.cv_results_))

train_m3v1_ls_y01_pred = m3v1_ls.predict(train_x01)
print(train_m3v1_ls_y01_pred)

test_m3v1_ls_y01_pred = m3v1_ls.predict(test_x01)
print(test_m3v1_ls_y01_pred)

# Display evaluation metrics
# R-sq
train_m3v1_ls_r2 = r2_score(train_y01, train_m3v1_ls_y01_pred)
test_m3v1_ls_r2 = r2_score(test_y01, test_m3v1_ls_y01_pred)

print(f'Train R-sq:\n{train_m3v1_ls_r2}')
print(f'Test R-sq:\n{test_m3v1_ls_r2}')

# RMSE
train_m3v1_ls_rmse = mean_squared_error(train_y01, train_m3v1_ls_y01_pred, squared=False)
test_m3v1_ls_rmse = mean_squared_error(test_y01, test_m3v1_ls_y01_pred, squared=False)

print(f'Train RMSE:\n{train_m3v1_ls_rmse}')
print(f'Test RMSE:\n{test_m3v1_ls_rmse}')

# End timer script
end_time = dt.datetime.today()
time_elapse = end_time - start_time
print(f'End Time = {end_time}')
print(f'Script Time = {time_elapse}')

coef_intercept = np.hstack((m3v1_ls.best_estimator_.named_steps["ls"].coef_,
                            m3v1_ls.best_estimator_.named_steps["ls"].intercept_))
#print(coef_intercept)

coef_intercept_df01 = pd.DataFrame(coef_intercept)
#display(coef_intercept_df01)

train_x01_col_names = list(train_x01.columns)
train_x01_col_names.append('intercept')

train_x01_col_names_df01 = pd.DataFrame(train_x01_col_names)
#display(train_x01_col_names_df01)

model_params = pd.concat([train_x01_col_names_df01, coef_intercept_df01], axis=1)
display(model_params)

### Lasso - Using `BayesSearchCV`

In [9]:
sklearn = SKLearn(entry_point="05_Modeling01_v2.py",
                  framework_version="0.20.0",
                  instance_type="ml.m5.xlarge",
                  role=role)

print(sklearn)

In [10]:
print(s3_train_path)

s3://sagemaker-us-east-1-657724983756/team_8_data/modeling_data/training


In [14]:
sklearn.fit({"train": s3_train_path})

INFO:sagemaker:Creating training-job with name: sagemaker-scikit-learn-2023-04-07-21-16-36-229


2023-04-07 21:16:42 Starting - Starting the training job...
2023-04-07 21:16:57 Starting - Preparing the instances for training...
2023-04-07 21:17:45 Downloading - Downloading input data...
2023-04-07 21:18:15 Training - Downloading the training image...
2023-04-07 21:18:25 Training - Training image download completed. Training in progress.2023-04-07 21:18:33,009 sagemaker-containers INFO     Imported framework sagemaker_sklearn_container.training
2023-04-07 21:18:33,012 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2023-04-07 21:18:33,021 sagemaker_sklearn_container.training INFO     Invoking user training script.
2023-04-07 21:18:33,224 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2023-04-07 21:18:33,235 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2023-04-07 21:18:33,247 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2023-04-07 21:18:3

UnexpectedStatusException: Error for Training job sagemaker-scikit-learn-2023-04-07-21-16-36-229: Failed. Reason: AlgorithmError: framework error: 
Traceback (most recent call last):
  File "/miniconda3/lib/python3.7/site-packages/sagemaker_containers/_trainer.py", line 84, in train
    entrypoint()
  File "/miniconda3/lib/python3.7/site-packages/sagemaker_sklearn_container/training.py", line 39, in main
    train(environment.Environment())
  File "/miniconda3/lib/python3.7/site-packages/sagemaker_sklearn_container/training.py", line 35, in train
    runner_type=runner.ProcessRunnerType)
  File "/miniconda3/lib/python3.7/site-packages/sagemaker_training/entry_point.py", line 100, in run
    wait, capture_error
  File "/miniconda3/lib/python3.7/site-packages/sagemaker_training/process.py", line 395, in run
    cwd=environment.code_dir,
  File "/miniconda3/lib/python3.7/site-packages/sagemaker_training/process.py", line 306, in check_error
    info=extra_info,
sagemaker_training.errors.E

In [ ]:
training_job_description = sklearn.jobs[-1].describe()
model_data_s3_uri = f'{training_job_description["OutputDataConfig"]["S3OutputPath"]}{training_job_description["TrainingJobName"]}/{"output/model.tar.gz"}'
print(training_job_description)
print(model_data_s3_uri)

In [ ]:
scikit_processing_job_name = processor.jobs[-1].describe()["ProcessingJobName"]
print(scikit_processing_job_name)

In [ ]:
from IPython.core.display import display, HTML

display(
    HTML(
        f'<b>Review <a target="blank" href="https://console.aws.amazon.com/sagemaker/home?region={region}#/processing-jobs/{scikit_processing_job_name}">Processing Job</a></b>'
    )
)

In [ ]:
from IPython.core.display import display, HTML

display(
    HTML(
        f'<b>Review <a target="blank" href="https://s3.console.aws.amazon.com/s3/buckets/{def_bucket}/{scikit_processing_job_name}/?region={region}&tab=overview">S3 Output Data</a> After The Processing Job Has Completed</b>'
    )
)

# Monitor the Processing Job

In [ ]:
running_processor = sagemaker.processing.ProcessingJob.from_processing_name(
    processing_job_name=scikit_processing_job_name, sagemaker_session=sagemaker_session
)

processing_job_description = running_processor.describe()

print(processing_job_description)

In [ ]:
running_processor.wait(logs=False)

## Release Resources

In [ ]:
%%html

<p><b>Shutting down your kernel for this notebook to release resources.</b></p>
<button class="sm-command-button" data-commandlinker-command="kernelmenu:shutdown" style="display:none;">Shutdown Kernel</button>
        
<script>
try {
    els = document.getElementsByClassName("sm-command-button");
    els[0].click();
}
catch(err) {
    // NoOp
}    
</script>

In [ ]:
%%javascript

try {
    Jupyter.notebook.save_checkpoint();
    Jupyter.notebook.session.delete();
}
catch(err) {
    // NoOp
}